In [89]:
import sys
sys.path.append('..')

In [90]:
from abc import ABC, abstractmethod
from random import randint

## Optimization problems and algorithms

### Optimization problem

An **optimization problem** is a problem that involves finding the best solution from a set of possible solutions, according to some criteria or objective. The goal is to either **maximize** or **minimize** a particular quantity (called the **objective function**) while satisfying certain constraints.

**A solution to an optimization problem is problem dependent.**

### Optimization algorithm

An **optimization algorithm** is a method used to find the best solution to an optimization problem. It is an interative algorithm that is able to return a solution at each iteration. The goal is to navigate the solution space, explore potential solutions, and ultimately identify the optimal or near-optimal solution according to the objective function.

**An optimization algorithm is not problem dependent.** You can use the exact same algorithm to solve any optimization problem, as long as you know how you can navigate the solution space.

### Solution

Let's start by implementig a generic `Solution` class.

What should a generic solution to an optimization problem look like?

Given that a solution to a problem `A` is different from a solution to a problem `B`, it is hard to characterize this generic solution. But there are some things that are common to any optimization problem solution.

Let's think of attributes and methods that are common to any optimization problem.

A solution must have the following attributes and methods:
- representation: How the solution is enconded. Having a representation makes it possible for an algorithm to manipulate and evaluate the solution
- fitness(): The function that determines how good a solution is
- random_initial_value(): If no representation is defined for the solution, it has to be possible to initialize it randomly


![Solution Class](images/solution.png)

Since these are problem-dependent, we can not implement them now. However, we can **enforce their implementation** in any subclass by defining this class as an **abstract class** with abstract methods. To do this, the class must inherit from `ABC`. Abstract methods have no implementation in the abstract class itself, but any subclass must implement them to allow object instantiation.

In [91]:
class Solution(ABC):
    def __init__(self, repr=None):
        if repr==None: #if not repr
            repr=self.random_init_value()
        self.repr=repr
    
    def __repr__(self):
        return self.repr

    @abstractmethod
    def fitness(self):
        pass
    
    @abstractmethod
    def random_init_value(self):
        pass



The final implementation is available in `library/solution.py`

## Hill Climbing

Hill Climbing is one of the most intuitive and immediate techniques for solving optimization problems. It works by iteratively improving fitness in a stepwise refinement process, using the concept of neighborhood to explore potential solutions.

### Pseudo-code

1. Initialize current solution (usually at random)
2. Repeat
   1. Get neighbors of current solution
   2. Find best neighbor
   3. If best neighbor is better or equal than current solution, replace current solution by best neighbor
   4. If current solution hasn't changed, break the cycle
3. Return current solution

### Algorithm Implementation

Let's implement this algorithm using python. The function that implements the algorithm should receive the following arguments:
- `initial solution`: an instance of a solution to an optimization problem
- `maximization`: boolean that indicates if we're solving a maximization or minimization problem
- `max_iter`: maximum number of interations. By default should be very big.

In [92]:
from copy import deepcopy

def hill_climbing(initial_solution, maximization, max_iter=99999,verbose=False):

    if not isinstance(initial_solution,Solution):
        raise ValueError("Initial solution is not solution")

    current_solution=initial_solution
    improved=True
    iter=1

    if verbose:
            print(f'Current solution: {initial_solution} with fitness {initial_solution.fitness()}')

    while improved:
        improved=False
        neighbors=current_solution.get_neighbors()
        for neighbor in neighbors:
            if maximization:
                if neighbor.fitness()>=current_solution.fitness():
                    current_solution=deepcopy(neighbor)
                    improved=True
            else:
                if neighbor.fitness()<=current_solution.fitness():
                    current_solution=deepcopy(neighbor)
                    improved=True
        if verbose:
            print(f'New Current solution: {current_solution} with fitness {current_solution.fitness()}')

        iter+=1
        if iter>=max_iter:
            break
    return current_solution




Notice that we assume that a solution has the following methods:
- `fitness()`
- `get_neighbors()`

Additionally, `get_neighbors()` must return a list of solutions that also implement these methods.

The final implementation is available in `library/algorithms/hill_climbing.py`

## IntBin Optimization Problem

**Description:** The IntBin problem consists of finding the integer with greatest number of 1's in its binary representation

**Search space:** Integers from 1 to 15.

**Representation:** Binary string of 4 digits representing the integer.

**Fitness function:** f(x)= Number of 1's in binary representation of x

**Goal:** Maximize f(x).

### IntBin Solution

Using the previously defined generic `Solution`, we can now define the `IntBinSolution` class that implements the fitness and random intial value methods for the IntBin problem.

This class represents a solution to the IntBin problem and **does not include any implementation related to the optimization algorithm that will be used to solve it**.

![IntBin Solution](images/intbin-solution.png)

In [93]:
class IntBinSolution(Solution):
    def __init__(self, repr=None):
        super().__init__(repr)
    
    def random_init_value(self):
        random=randint(1,15)
        return format(random,"04b")
    
    def fitness(self):
        return self.repr.count("1")
    

Let's test it.

In [94]:
# Intialize with representation
solution = IntBinSolution('0001')

# Initialize with random representation
solution_random = IntBinSolution()

print(f'Solution {solution} with fitness {solution.fitness()}')
print(f'Random solution {solution_random} with fitness {solution_random.fitness()}')

Solution 0001 with fitness 1
Random solution 0010 with fitness 1


### Solving IntBin with Hill Climbing

To solve the IntBin problem using Hill Climbing, we need to define how to navigate the solution space. In hill climbing, the search space is navigated with the concept of neighborhood.

The algorithm requires the solution to have a `get_neighbors()` method. Therefore, we can **extend** the `IntBinSolution` and create a `IntBinHillClimbingSolution` that implements the `get_neighbors()` method.

![IntBin Hill Climbing Solution Inheritance](images/intbin-hillclimbing-solution.png)

There are two options to get the neighbors of a solution:
- Option 1 - Integer neighborhood: Each integer x has at most two neighbors: x-1 and x+1, except for boundaries (1 and 15).
- Option 2 - Bit flip neighborhood: Each binary representation of an integer x has as neighbors any other binary with a bit flipped.

#### Option 1 - Integer neighborhood: Each integer x has at most two neighbors: x-1 and x+1, except for boundaries (1 and 15).

Let's create a `IntBin_IntNeighborhood_HillClimbingSolution` class that inherits from `IntBinSolution` and implements the `get_neighbors()` method.

We also need to make sure that this method return a list of IntBin solutions that also implement `get_neighbors()`, meaning it should return a list of `IntBin_IntNeighborhood_HillClimbingSolution` instances.

In [95]:
class IntBin_IntNeighborhood_HillClimbingSolution(IntBinSolution):
    def __init__(self, repr=None):
        super().__init__(repr)   #inutil
    
    def get_neighbors(self):
        bin_repr=self.repr
        integer_repr=int(bin_repr,2)

        if integer_repr==15:
            neighbor_bin_repr= format(14,"04b")
            return [IntBin_IntNeighborhood_HillClimbingSolution(repr=neighbor_bin_repr)]
        
        if integer_repr==1:
            neighbor_bin_repr= format(2,"04b")
            return [IntBin_IntNeighborhood_HillClimbingSolution(repr=neighbor_bin_repr)]
        
        else:
            neighbor1_bin_repr=format(integer_repr-1, "04b")
            neighbor2_bin_repr=format(integer_repr+1, "04b")
            return [IntBin_IntNeighborhood_HillClimbingSolution(repr=neighbor1_bin_repr),IntBin_IntNeighborhood_HillClimbingSolution(repr=neighbor2_bin_repr)]

Let's test it

In [96]:
# Initialize a random solution
solution = IntBin_IntNeighborhood_HillClimbingSolution('1010')
print('Solution', solution)

neighbors = solution.get_neighbors()
print('Neighbors:')
for neighbor in neighbors:
    print(neighbor)

Solution 1010
Neighbors:
1001
1011


We can now apply the HillClimbing algorithm to the IntBin problem by passing it an initial solution

In [100]:
initial_solution=IntBin_IntNeighborhood_HillClimbingSolution()
best_sol=hill_climbing(initial_solution,maximization=True,verbose=True)
print('Best sol',best_sol)

Current solution: 1100 with fitness 2
New Current solution: 1101 with fitness 3
New Current solution: 1110 with fitness 3
New Current solution: 1111 with fitness 4
New Current solution: 1111 with fitness 4
Best sol 1111


Let's see if the final solution changes with multiple runs

In [101]:
for i in range(10):
    initial_solution=IntBin_IntNeighborhood_HillClimbingSolution()
    best_sol=hill_climbing(initial_solution,maximization=True)
    print(best_sol)

1111
1011
1111
1011
1111
0111
0011
1111
0011
1111


Different runs produce different solutions, and not always the global optimum (1111) is found

In the next notebook, we will implement the IntBin problem using Hill Climbing, adopting Option 2 (Bit Flip Neighborhood) to explore the solution space and analyse the differences.